In [16]:
import pandas as pd

# Dataset

In [128]:
url = 'https://proai-datasets.s3.eu-west-3.amazonaws.com/formula1_data.csv'

In [129]:
df = pd.read_csv(url)

In [130]:
df.head()

,Driver,Team,Race,Country,Position
0,Hamilton,McLaren,Melbourne,Australia,1
1,Massa,Ferrari,Melbourne,Australia,0
2,Raikkonen,Ferrari,Melbourne,Australia,8
3,Kubica,BMW,Melbourne,Australia,0
4,Alonso,Renault,Melbourne,Australia,4


In [131]:
# check if there are some null values to handle
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Driver    180 non-null    object
 1   Team      180 non-null    object
 2   Race      180 non-null    object
 3   Country   180 non-null    object
 4   Position  180 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 7.2+ KB


In [132]:
points_system = {1: 10, 2: 8, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1}

# Functions

## individual_performances

In [133]:
def individual_performances():


    '''
    This function takes a driver's name as input and returns a list containing:
    - the total points accumulated by the driver during the season;
    - the number of wins (first-place finishes);
    - the number of podiums (finishes in the top 3).

    If the driver's name is not found in the dataset, the function prompts the user to enter a valid name.
    '''


    # User input
    while True:
        name = input('\nInsert driver name: ')
        if name in df['Driver'].values:
            break
        print('Driver not found. Please insert a valid driver name.')

    # Filter the dataset for the selected driver
    driver_data = df[df['Driver'] == name]

    # Calculate the total points by mapping positions to the points system
    # fillna() is added to handle 9th position and beyond
    total_points = driver_data['Position'].map(points_system).fillna(0).sum()

    # Count the number of wins (1st place finishes)
    wins = (driver_data['Position'] == 1).sum()

    # Count the number of podiums (1st, 2nd, or 3rd place finishes)
    podiums = (driver_data['Position'].isin([1, 2, 3])).sum()

    return [int(total_points), wins, podiums]

In [134]:
# Ultimate check
individual_performances()


Insert driver name: Marcello
Driver not found. Please insert a valid driver name.

Insert driver name: Hamilton


[98, 5, 10]

## individual_ranking

In [135]:
def individual_ranking():


    '''
    This function generates a dictionary with driver names as keys and their total points as values,
    sorts the drivers by their total points, and saves the ranking to a text file.
    '''


    # Calculate total points for each driver
    # Using apply with lambda instead of map because we are working with groups
    # fillna() is added to handle 9th position and beyond
    driver_points = df.groupby('Driver')['Position'].apply(
        lambda positions: positions.map(points_system).fillna(0).sum()
    ).to_dict()

    # Sort the drivers by their total points in descending order
    sorted_drivers = dict(sorted(driver_points.items(), key=lambda item: item[1], reverse=True))

    # Save the individual ranking to a text file
    with open('Drivers_Standings_2008.txt', 'w') as file:
        file.write('Drivers Standings 2008 Formula 1\n')
        for driver, points in sorted_drivers.items():
            file.write(f'{driver}: {int(points)}\n')  # Convert points to int before writing

    print('Individual ranking have been saved to "Drivers_Standings_2008.txt"')

In [136]:
individual_ranking()

Individual ranking have been saved to "Drivers_Standings_2008.txt"


In [137]:
# Read
with open('Drivers_Standings_2008.txt', 'r') as file:
    print(file.read())

Drivers Standings 2008 Formula 1
Hamilton: 98
Massa: 97
Kubica: 75
Raikkonen: 75
Alonso: 61
Heidfeld: 60
Kovalainen: 53
Vettel: 35
Trulli: 31
Glock: 25



## team_ranking

In [138]:
def team_ranking(file_path):


    '''
    This function generates a dictionary with team names as keys and their total points as values.
    It reads the driver standings from the specified file and calculates the total points
    for each constructor by summing the points obtained by its drivers.

    The function then returns the sorted team rankings.

    Parameters:
    - file_path (str): Path to the file containing driver standings.
    '''


    # Read the driver standings from the specified file
    driver_points = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # Skip the header line

        for line in lines:
            driver, points = line.strip().split(': ')
            driver_points[driver] = int(float(points))  # Convert float to int to remove ".0"

    # Merge driver points with the team column
    df['Total Points'] = df['Driver'].map(driver_points)

    # Calculate total points for each team by summing up the points of its drivers
    team_points = df.groupby('Team')['Total Points'].sum().astype(int).to_dict()

    # Sort the teams by their total points in descending order
    sorted_teams = dict(sorted(team_points.items(), key=lambda item: item[1], reverse=True))

    return sorted_teams  # Return the sorted dictionary

In [139]:
team_ranking("/content/Drivers_Standings_2008.txt")

{'Ferrari': 3096,
 'McLaren': 2718,
 'BMW': 2430,
 'Renault': 1098,
 'Toyota': 1008,
 'Toro Rosso': 630}